# 07 - Prediction Serving

The purpose of the notebook is to show how to use the deployed model for online and batch prediction.
The notebook covers the following tasks:
1. Test the endpoints for online prediction.
2. Use the uploaded custom model for batch prediction.
3. Run a the batch prediction pipeline using `Vertex Pipelines`.

## Setup

### Import libraries

In [ ]:
import os
from datetime import datetime
import tensorflow as tf

from google.cloud import aiplatform as vertex_ai

### Setup Google Cloud project

In [ ]:
PROJECT = '' # Change to your project id.
REGION = '' # Change to your region.
BUCKET = ''

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP project id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn't exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)

### Set configurations

In [ ]:
VERSION = 'v01'
DATASET_DISPLAY_NAME = 'tfdf-rules'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
ENDPOINT_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier'

## 1. Making Online Predictions


In [ ]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=BUCKET
)

endpoint_name = vertex_ai.Endpoint.list(
    filter=f'display_name={ENDPOINT_DISPLAY_NAME}', 
    order_by="update_time")[-1].gca_resource.name

endpoint = vertex_ai.Endpoint(endpoint_name)

In [ ]:
test_instances = [  
    {
        "feature_1": [0],
        "feature_2": [1],
        "feature_3": [0],
        "feature_4": [0],
        "feature_5": [1],
    }
]

In [ ]:
predictions = endpoint.predict(test_instances).predictions

for prediction in predictions:
    print(prediction)

In [ ]:
import pandas as pd
import numpy as np
CSV_DATA_PATH = 'data/data.csv'
df = pd.read_csv(CSV_DATA_PATH)

index_of_label = np.argmax(predictions) - 1
print(f'index_of_label: {index_of_label}')

print(df)
df.iloc[index_of_label]

In [ ]:
# Explanations aren't supported on this model.
# explanations = endpoint.explain(test_instances).explanations

# for explanation in explanations:
#     print(explanation)

### (Optional) Build the ML container image

This is the `TFX` runtime environment for the training pipeline steps.

In [ ]:
!echo $TFX_IMAGE_URI

In [ ]:
!gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8

### Compile pipeline

In [ ]:
from src.tfx_pipelines import runner

pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
pipeline_definition = runner.compile_prediction_pipeline(pipeline_definition_file)

### Submit run to Vertex Pipelines

In [ ]:
from kfp.v2.google.client import AIPlatformClient

pipeline_client = AIPlatformClient(
    project_id=PROJECT, region=REGION)
                 
pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file
)